# Testing single-nucleon momentum distributions

__Author:__ A. J. Tropiano [atropiano@anl.gov]<br/>
__Date:__ September 7, 2023

This notebook serves as a testbed for calculating momentum distributions using mean field approximations for initial and final states and applying SRG transformations to the operator.

_Last update:_ October 31, 2023

In [1]:
# Python imports
from functools import partial
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.special import spherical_jn
from sympy.physics.quantum.cg import CG
import vegas

In [2]:
# JAX imports
from jax import jit, vmap
from jax.config import config
import jax.numpy as jnp
from jax.numpy.linalg import norm
from jax.scipy.special import sph_harm

In [3]:
# Other imports
from scripts.integration import momentum_mesh

## Set-up

In [4]:
# Enable double precision
config.update("jax_enable_x64", True)

# Functions

In [ ]:
@jit
def kronecker_delta(x, y):
    """Kronecker \delta function: \delta_{x,y}."""
    
    return jnp.asarray(x == y, dtype=int)

In [ ]:
# TESTING
print(kronecker_delta(1,1))
print(kronecker_delta(1.5,-0))

In [ ]:
@jit
def build_vector(k, theta, phi):
    """
    Build a vector from input spherical coordinates.

    Parameters
    ----------
    k : float
        Magnitude of the vector.
    theta : float
        Polar angle of the vector in the range [0, \pi].
    phi : float
        Azimuthal angle of the vector in the range [0, 2\pi].

    Returns
    -------
    k_vector : 1-D jnp.ndarray
        Output vector with shape (3,1).

    """

    k_vector = jnp.array([k * jnp.sin(theta) * jnp.cos(phi),
                          k * jnp.sin(theta) * jnp.sin(phi),
                          k * jnp.cos(theta)])

    return k_vector

In [ ]:
@jit
def get_vector_components(k_vector):
    """
    Get the spherical coordinates from an input vector.

    Parameters
    ----------
    k_vector : 1-D jnp.ndarray
        Input vector with shape (3,1).

    Returns
    -------
    k : float
        Magnitude of the vector.
    theta : float
        Polar angle of the vector in the range [0, \pi].
    phi : float
        Azimuthal angle of the vector in the range [0, 2\pi].

    """

    k = norm(k_vector)
    theta = jnp.arccos(k_vector[2]/k)
    phi = jnp.arctan2(k_vector[1], k_vector[0])

    return k, theta, phi

In [ ]:
# TESTING
k_vector = build_vector(0.5, 0.1, 0.05)
print(k_vector)
k, theta, phi = get_vector_components(k_vector)
print(k, theta, phi)

# Clebsch-Gordan function and class

In [ ]:
class ClebschGordan:
    
    def __init__(self, save=False):
        
        # Save array of Clebsch-Gordan coefficients array as .npz file
        if save:
            
            jmax = 6.5  # This should work for nuclei <= Pb208
            self.cg_array, self.N_j = self.compute_clebsch_gordan_array(jmax)
            jnp.save('cg_array', self.cg_array)
            
        # Load Clebsch-Gordan coefficients array
        else:
            
            self.cg_array = jnp.load('cg_array.npy')
            self.N_j = self.cg_array.shape[0]
        
    def compute_clebsch_gordan_array(self, j_max):
        """Calculate Clebsch-Gordan coefficients for combinations of j and m_j
        up to j_max.
        """

        j_array = np.arange(0, j_max + 1/2, 1/2)
        N_j = j_array.size
    
        # 0, 1/2, 1, ..., J, -1/2, -1, ..., -J
        m_array = np.concatenate((j_array, -j_array[1:]))
        N_m = m_array.size
    
        cg_array = np.zeros((N_j, N_j, N_j, N_m, N_m, N_m))
    
        for i, j_1 in enumerate(j_array):
            m_1_array = np.arange(-j_1, j_1+1)
            for j, j_2 in enumerate(j_array):
                m_2_array = np.arange(-j_2, j_2+1)
                j_3_array = np.arange(np.abs(j_1-j_2), j_1+j_2+1)
                for k, j_3 in enumerate(j_array):
                    m_3_array = np.arange(-j_3, j_3+1)
                    for l, m_1 in enumerate(m_array):
                        for m, m_2 in enumerate(m_array):
                            for n, m_3 in enumerate(m_array):
                            
                                selection_rules = (
                                    np.any(j_3 == j_3_array)
                                    and np.any(m_1 == m_1_array)
                                    and np.any(m_2 == m_2_array)
                                    and np.any(m_3 == m_3_array)
                                    and m_1 + m_2 == m_3
                                )

                                if selection_rules:

                                    cg_array[i, j, k, l, m, n] = float(
                                        CG(j_1,m_1,j_2,m_2,j_3,m_3).doit()
                                    )
    
        return jnp.array(cg_array, dtype=jnp.float64), N_j

    @partial(jit, static_argnums=(0,))
    def m_index(self, m):
        """Returns the unique index associated with m."""
        
        return jnp.asarray(
            jnp.abs(m / 0.5) + jnp.heaviside(-m, 0) * (self.N_j - 1), dtype=int
        )

    @partial(jit, static_argnums=(0,))
    def get_cg(self, j1, m1, j2, m2, j3, m3):
        """Clebsch-Gordan coefficient < j1 m1 j2 m2 | j3 m3 >."""

        i_j1 = jnp.asarray(j1 / 0.5, dtype=int)
        i_m1 = self.m_index(m1)
        i_j2 = jnp.asarray(j2 / 0.5, dtype=int)
        i_m2 = self.m_index(m2)
        i_j3 = jnp.asarray(j3 / 0.5, dtype=int)
        i_m3 = self.m_index(m3)
        
        return self.cg_array[i_j1, i_j2, i_j3, i_m1, i_m2, i_m3]
    
    @partial(jit, static_argnums=(0,))
    def vmap_get_cg(self, j1, m1, j2, m2, j3, m3):

        return vmap(self.get_cg)(j1, m1, j2, m2, j3, m3)

In [ ]:
# # Save Clebsch-Gordan array as .npy file
# cg = ClebschGordan(save=True)

In [ ]:
# TESTING
# < 1/2 -1/2 1/2 1/2 | 1 0 > should give -1/\sqrt(2)
cg = ClebschGordan()
print(cg.get_cg(1/2, -1/2, 1/2, 1/2, 1, 0))
j_array = jnp.array([1/2, 1/2])
m1_array = jnp.array([1/2, -1/2])
m2_array = jnp.array([1/2, 1/2])
J_array = jnp.array([1, 1])
M_array = jnp.array([1, 0])
print(cg.vmap_get_cg(j_array, m1_array, j_array, m2_array, J_array, M_array))

# Woods-Saxon class

In [ ]:
# TODO: Equal filling approximation
class WoodsSaxon:
    """DOCSTRING:
        major attributes: occupied_states, mapping_array, wave_functions
        major methods: mapping, spinor_spherical_harmonic, psi
    """
    
    def __init__(
        self, nucleus_name: str, Z: int, N: int, clebsch_gordan: callable,
        kmax=10.0, kstep=0.1
    ):
        
        # Set instance attributes
        self.woods_saxon_directory = f"../data/woods_saxon/{nucleus_name}/"
        self.clebsch_gordan = clebsch_gordan
        
        # Order single-particle states with lowest energy first
        occupied_states = self.order_sp_states(Z, N)
        
        # Set-up mapping for [n, l, j, m_j, m_t] -> single integer index
        self.set_mapping(occupied_states)
        
        # Set-up momentum points for single-particle wave functions
        self.k_array = np.arange(0.0, kmax + kstep, kstep)
        N_k = self.k_array.size

        # Set JAX array of occupied momentum-space wave functions \phi_\alpha(k)
        self.wave_functions = jnp.zeros((Z + N, N_k), dtype=complex)
        for i, occupied_state in enumerate(occupied_states):
            n, l, j, m_j, m_t = occupied_state  # Degenerate in m_j
            phi_k_array = self.compute_wf_kspace(n, l, j, m_t)
            self.wave_functions = self.wave_functions.at[i].set(phi_k_array)

    def order_sp_states(self, Z, N):
        """Keep track of all s.p. states and occupied s.p. states"""

        # Occupied single-particle states < E_F
        occupied_states = []
        proton_count = 0
        neutron_count = 0
        
        # File with details of the orbitals
        ws_file = self.woods_saxon_directory + "ws_log"
    
        # Order single-particle states using the ws_log file
        with open(ws_file, 'r') as f:
            for line in f:
                unit = line.strip().split()
                
                # Protons
                if len(unit) > 0 and unit[0] == '1':
                    j = int(unit[3])/2
                    for m_j in np.arange(-j, j+1, 1):
                        # (n, l, j, m_j, m_t) tuple
                        sp_state = (
                            int(unit[1])+1, int(unit[2]), j, m_j, 1/2
                        )
                        if proton_count < Z:
                            occupied_states.append(sp_state)
                            # Add up filled proton states
                            proton_count += 1
                
                # Neutrons
                elif len(unit) > 0 and unit[0] == '2':
                    j = int(unit[3])/2
                    for m_j in np.arange(-j, j+1, 1):
                        # (n, l, j, m_j, m_t) tuple
                        sp_state = (
                            int(unit[1])+1, int(unit[2]), j, m_j, -1/2
                        )
                        if neutron_count < N:
                            occupied_states.append(sp_state)
                            # Add up filled neutron states
                            neutron_count += 1
                            
        return occupied_states
    
    def get_wf_rspace(self, n, l, j, m_t, print_norm=False):
        """Single-particle wave function in coordinate space."""
        
        if m_t == 1/2:
            file_name = f"p.n{int(n-1)}.l{int(l)}.j{int(2*j)}.orb"
        elif m_t == -1/2:
            file_name = f"n.n{int(n-1)}.l{int(l)}.j{int(2*j)}.orb"
            
        data = np.loadtxt(self.woods_saxon_directory + file_name)
        
        r_array = data[:, 0]  # fm
        u_array = data[:, 1]  # fm^-1/2
        
        # Normalization: \int dr |u(r)|^2 = 1
        if print_norm:
            normalization = np.trapz(u_array ** 2)
            print(f"Coordinate space normalization = {normalization}.")

        return r_array, u_array

    def compute_wf_kspace(self, n, l, j, m_t):
        """Single-particle wave function in momentum space."""
        
        # Load Woods-Saxon orbital
        r_array, u_array = self.get_wf_rspace(n, l, j, m_t)
        dr = r_array[2] - r_array[1]  # Step-size in r
    
        # Get meshgrids of k and r for integration over r
        k_grid, r_grid = np.meshgrid(self.k_array, r_array, indexing='ij')
        _, u_grid = np.meshgrid(self.k_array, u_array, indexing='ij')

        integrand = r_grid * u_grid * spherical_jn(l, k_grid * r_grid) * dr
        
        return 1j ** (-l) * np.sqrt(2/np.pi) * np.sum(integrand, axis=-1)

    def set_mapping(self, occupied_states):
        """Maps s.p. state to corresponding index in wave functions array."""

        # Get maximum n, l, and j
        self.occupied_states_array = np.array(occupied_states)
        n_max = np.amax(self.occupied_states_array[:, 0])
        l_max = np.amax(self.occupied_states_array[:, 1])
        self.j_max = np.amax(self.occupied_states_array[:, 2])
        
        # Size of each component of the mapping array
        n_size = int(n_max + 1)
        l_size = int(l_max + 1)
        j_size = int(self.j_max + 1/2)
        m_j_size = int(2*j_size)

        # Initialize mapping array
        self.mapping_array = jnp.zeros((n_size, l_size, j_size, m_j_size, 2),
                                       dtype=int)
            
        # Loop over occupied states and fill in index i for state
        for i, occupied_state in enumerate(occupied_states):
            
            n, l, j, m_j, m_t = occupied_state
            n_index = int(n-1)
            l_index = int(l)
            j_index = int(j - 1/2)
            m_j_index = int(self.j_max + j)
            if m_t == 1/2:
                m_t_index = 0
            elif m_t == -1/2:
                m_t_index = 1

            self.mapping_array = self.mapping_array.at[
                n_index, l_index, j_index, m_j_index, m_t_index
            ].set(i)
    
    @partial(jit, static_argnums=(0,))
    def mapping(self, n, l, j, m_j, m_t):
        """Returns the unique index i of the occupied state given orbital."""
        
        n_index = jnp.asarray(n-1, dtype=int)
        l_index = jnp.asarray(l, dtype=int)
        j_index = jnp.asarray(j - 1/2, dtype=int)
        m_j_index = jnp.asarray(self.j_max + j, dtype=int)
        m_t_index = jnp.where(m_t == 1/2, 0, 1)
        
        return self.mapping_array[n_index, l_index, j_index, m_j_index,
                                  m_t_index]

    @partial(jit, static_argnums=(0,))
    def spinor_spherical_harmonic(self, theta, phi, sigma, l, j, m_j):
        """Spinor spherical harmonic for a s.p. state described by the quantum
        numbers j, m_j, l, and s=1/2.
        """
        
        # Spinor indexed by \sigma \eta_{m_s}^(\sigma) = \delta_{m_s, \sigma}
        m_s = sigma
    
        # m_l must be fixed since m_j and m_s are determined
        m_l = m_j - m_s
        
        # Make sure that |m_l| <= l
        condition = jnp.abs(m_l) <= l
        
        return jnp.where(
            condition,
            (
                self.clebsch_gordan.get_cg(l, m_l, 1/2, m_s, j, m_j)
                * sph_harm(
                    jnp.array([m_l], dtype=int), jnp.array([l], dtype=int), phi,
                    theta, n_max=10
                )
            ),
            0+0j
        )
    
    @partial(jit, static_argnums=(0,))
    def psi(self, k, theta, phi, sigma, tau, n, l, j, m_j, m_t):
        """Single-particle wave function \psi_\alpha(k_vector; \sigma, \tau)."""

        # Get \phi_\alpha(k)
        index = self.mapping(n, l, j, m_j, m_t)
        phi_k_array = self.wave_functions[index]
        
        # Interpolate to point k
        phi_k = jnp.interp(k, self.k_array, phi_k_array)
        
        # Calculate spinor spherical harmonic
        Y_jml = self.spinor_spherical_harmonic(theta, phi, sigma, l, j, m_j)
    
        # Isospinor indexed by \tau \chi_{m_t}(\tau)
        chi_tau = kronecker_delta(tau, m_t)

        return phi_k * Y_jml * chi_tau
    
    # TODO: Test how to do a function that returns 3-D array starting with function that depends on scalars
    # (Focus on getting working code first -> scaling is problematic with \delta U's not IPM!)
    @partial(jit, static_argnums=(0,))
    def vmap_psi(self, k, theta, phi, sigma, tau, n, l, j, m_j, m_t):

        return vmap(
            self.psi,
            in_axes=(0, 0, 0, None, None, 1, 1, 1, 1, 1),
            out_axes=(0,1)
        )(k, theta, phi, sigma, tau, n, l, j, m_j, m_t)

In [ ]:
# TESTING
clebsch_gordan = ClebschGordan()
woods_saxon = WoodsSaxon('C12', 6, 6, clebsch_gordan)

psi = woods_saxon.psi(0.5, 0.1, 0.05, 1/2, 1/2, 1, 0, 1/2, 1/2, 1/2)
print(psi)

In [ ]:
def get_sp_wave_functions(woods_saxon, kmax, kmid, ntot):
    """Set interpolating functions for s.p. wave functions \phi."""
    
    occ_states = woods_saxon.occ_states

    phi_functions = {}
    for sp_state in occ_states: 
        key = (sp_state.n, sp_state.l, sp_state.j, sp_state.m_t)
        phi_functions[key] = woods_saxon.get_wf_kspace(sp_state, kmax, kmid,
                                                       ntot, interpolate=True)
            
    return phi_functions

In [ ]:
from single_particle_states import WoodsSaxon as WS
from test_momentum_distribution_script import psi as pssi
from clebsch_gordan import compute_clebsch_gordan_table as ccgt

ws = WS('C12', 6, 6)
cg_table = ccgt(3)
phi_functions = get_sp_wave_functions(ws, 10.0, 2.0, 120)
psi = pssi(1, 0, 1/2, 1/2, 1/2, 0.5, 0.1, 0.05, 1/2, 1/2, cg_table,
           phi_functions)
print(psi)

In [5]:
A = 208
test = jnp.zeros((A, 5))

# $\delta U$ matrix element class

In [ ]:
class DeltaUMatrixElement:
    
    def __init__(self):
        pass

# Momentum distribution class

In [ ]:
class DeltaUIntegrand:
    
    def __init__(
        self, tau: float, q: float, woods_saxon: callable,
        delta_U_matrix_element: callable
    ):
        
        # Set instance attributes
        self.tau = tau
        self.q = q
        self.woods_saxon = woods_saxon
        self.delta_U_matrix_element = delta_U_matrix_element
        
        # Arrays of occupied states of Woods-Saxon [(n, l, j, m_j, m_t), ...]
        n_array = woods_saxon.occupied_states_array[:, 0]
        l_array = woods_saxon.occupied_states_array[:, 1]
        j_array = woods_saxon.occupied_states_array[:, 2]
        m_j_array = woods_saxon.occupied_states_array[:, 3]
        m_t_array = woods_saxon.occupied_states_array[:, 4]
        
        # Set meshgrid of quantum numbers
        # THIS WILL CAUSE AN ERROR
        # NEED TO PACKAGE THINGS BETTER
        (self.n_alpha, self.l_alpha, self.j_alpha, self.m_j_alpha,
         self.m_t_alpha, self.n_beta, self.l_beta, self.j_beta, self.m_j_beta,
         self.m_t_beta, self.sigma_1, self.sigma_2, self.sigma, self.sigmap,
         self.tau_1, self.tau_2, self.taup ) = (
            jnp.meshgrid(
                n_array, l_array, j_array, m_j_array, m_t_array, n_array,
                l_array, j_array, m_j_array, m_t_array, spins, spins, spins,
                spins, spins, spins, spins, indexing='ij'
            )
        )
    
    @partial(jit, static_argnums=(0,))
    def __call__(self, x_array):
        """Sum over all s.p. quantum numbers and return integrand."""
        
        # Multi-dimensional array w.r.t. many s.p. quantum numbers
        integrand_array = self.vmap_evaluate(
            x_array, self.alpha, self.beta, self.sigma_1, self.sigma_2,
            self.sigma, self.sigmap, self.tau_1, self.tau_2, self.taup
        )
        
        # __call__ should sum over \alpha
        # previous method should sum over \beta
        # previous method should sum over configurations of \sigma
        # previous method should sum over configurations of \tau
        
        # vmap_evaluate method: vectorizes isospin
        # method sum_over_isospin: call jnp.sum(vmap_evaluate) to sum over isospin
        # vmap_sum_over_isospin: vectorizes spin
        # method sum_over_spin: call jnp.sum(vmap_sum_over_isospin)
        # vmap_sum_over_spin: vectorizes \beta
        # method sum_over_beta: call jnp.sum(vmap_sum_over_spin)
        # vmap_sum_over_beta: vectorizes \alpha
        # method __call__: call jnp.sum(vmap_sum_over_beta)
        
        # Performs all s.p. quantum number sums
        return jnp.sum(integrand_array)
    
    @partial(jit, static_argnums=(0,))
    def vmap_alpha(self, x_array, alpha):
        return vmap(self.sum_over_beta, in_axes=(None, 0))(alpha)(
            ...
        )
    
    @partial(jit, static_argnums=(0,))
    def sum_over_beta(self, x_array, alpha):
        beta = None
        return jnp.sum(self.vmap_beta(x_array, alpha, beta))
    
    @partial(jit, static_argnums=(0,))
    def vmap_beta(self, x_array, alpha, beta):
        return vmap(self.sum_over_spin, in_axes=(None, None, 0))(
            ...
        )
    
    @partial(jit, static_argnums=(0,))
    def sum_over_spin(self, x_array, alpha, beta):
        spin = None
        return jnp.sum(self.vmap_spin(x_array, alpha, beta, spin))
    
    @partial(jit, static_argnums=(0,))
    def vmap_spin(self, x_array, alpha, beta, spin):
        return vmap(self.sum_over_isospin, in_axes=(None, None, None, 0))(
            ...
        )
    
    @partial(jit, static_argnums=(0,))
    def sum_over_isospin(self, x_array, alpha, beta, spin):
        isospin = None
        return jnp.sum(self.vmap_isospin(x_array, alpha, beta, spin, isospin))
    
    @partial(jit, static_argnums=(0,))
    def vmap_isospin(self, x_array, alpha, beta, spin, isospin):
        return vmap(self.evaluate, in_axes=(None, None, None, None, 0))(
            ...
        )
    
    @partial(jit, static_argnums=(0,))
    def evaluate(self, x_array, alpha, beta, spin, isospin):
        """Evaluate integrand for particular point in momenta and configuration
        of quantum numbers. This method assumes sampling over the following:
        
            k, theta_k, phi_k -> vector momenta k,
            K, theta_K, phi_K -> vector momenta K,
            
        where x_array contains each variable meaning x_array.shape = (6,).
        """

        # Choose z-axis to be along q_vector
        q_vector = jnp.array([0, 0, self.q])
        
        # Relative momenta k
        k, theta_k, phi_k = x_array[0:3]
        k_vector = build_vector(k, theta_k, phi_k)
        
        # C.o.M. momenta K
        K, theta_K, phi_K = x_array[3:6]
        K_vector = build_vector(K, theta_K, phi_K)

        # Calculate vector q - K/2
        qK_vector = q_vector - K_vector/2
        qK, theta_qK, phi_qK = get_vector_components(qK_vector)
        
        # Calculate vector k_1 = K/2 + k
        k1_vector = K_vector/2 + k_vector
        k1, theta_k1, phi_k1 = get_vector_components(k1_vector)
        
        # Calculate vector k_2 = K/2 - k
        k2_vector = K_vector/2 - k_vector
        k2, theta_k2, phi_k2 = get_vector_components(k2_vector)
        
        # Calculate vector K - q
        Kq_vector = K_vector - q_vector
        Kq, theta_Kq, phi_Kq = get_vector_components(Kq_vector)
        
        # Calculate the Jacobian determinant
        jacobian = k ** 2 * jnp.sin(theta_k) * K ** 2 * jnp.sin(theta_K)
  
        # Plane-wave matrix elements of \delta U and \delta U^\dagger
        delta_U_plane_wave = self.delta_U_matrix_element(
            k, theta_k, phi_k, qK, theta_qK, phi_qK, sigma_1, sigma_2,
            sigma, sigmap, tau_1, tau_2, self.tau, taup, hc=False
        )
        delta_U_dag_plane_wave = self.delta_U_matrix_element(
            qK, theta_qK, phi_qK, k, theta_k, phi_k, sigma, sigmap, sigma_1,
            sigma_2, self.tau, taup, tau_1, tau_2, hc=True
        )
        
        # \psi_\alpha(K/2+k; \sigma_1, \tau_1)
        psi_alpha_1 = self.woods_saxon.psi(
            k1, theta_k1, phi_k1, sigma_1, tau_1, n_alpha, l_alpha, j_alpha,
            m_j_alpha, m_t_alpha
        )
        # \psi_\beta(K/2-k; \sigma_2, \tau_2)
        psi_beta_2 = self.woods_saxon.psi(
            k2, theta_k2, phi_k2, sigma_2, tau_2, n_beta, l_beta, j_beta,
            m_j_beta, m_t_beta
        )
        
        # \psi_\alpha(q; \sigma, \tau)
        psi_alpha_q = self.woods_saxon.psi(
            self.q, 0.0, 0.0, sigma, tau, n_alpha, l_alpha, j_alpha, m_j_alpha,
            m_t_alpha
        )
        # \psi_\beta(K-q; \sigma', \tau')
        psi_beta_Kq = self.woods_saxon.psi(
            Kq, theta_Kq, phi_Kq, sigmap, taup, n_beta, l_beta, j_beta,
            m_j_beta, m_t_beta
        )
        
        # \psi_\alpha(K-q; \sigma', \tau')
        psi_alpha_Kq = self.woods_saxon.psi(
            Kq, theta_Kq, phi_Kq, sigmap, taup, n_alpha, l_alpha, j_alpha,
            m_j_alpha, m_t_alpha
        )
        # \psi_\beta(q; \sigma, \tau)
        psi_beta_q = self.woods_saxon.psi(
            self.q, 0.0, 0.0, sigma, tau, n_beta, l_beta, j_beta, m_j_beta,
            m_t_beta
        )
        
        # \delta U term
        delta_U_term = (
            delta_U_plane_wave * jnp.conj(psi_alpha_1) * jnp.conj(psi_beta_2)
            * (psi_beta_Kq * psi_alpha_q - psi_alpha_Kq * psi_beta_q)
        )
        
        # \delta U^\dagger term
        delta_U_dag_term = (
            delta_U_dag_plane_wave * psi_alpha_1 * psi_beta_2
            * (
                np.conj(psi_beta_Kq) * np.conj(psi_alpha_q)
                - np.conj(psi_alpha_Kq) * np.conj(psi_beta_q)
            )
        )

        # Add together for full integrand
        integrand = 1/2 * jacobian * (delta_U_term + delta_U_dag_term)
        
        return integrand.real
    
    @partial(jit, static_argnums=(0,))
    def vmap_evaluate(
            self, x_array, alpha, beta, sigma_1, sigma_2, sigma, sigmap, tau_1,
            tau_2, taup
    ):
        """Vectorized version of the previous method."""
        
        return vmap(self.evaluate, in_axes=(None, 0, 0, 0, 0, 0, 0, 0, 0, 0))(
            x_array, alpha, beta, sigma_1, sigma_2, sigma, sigmap, tau_1, tau_2,
            taup
        )

In [ ]:
class DeltaU2Integrand:
    
    def __init__(self):
        pass
    
    def __call__(self):
        pass

In [ ]:
class MomentumDistribution:
    """DOCSTRING"""
    
    def __init__(
        self, nucleus_name, Z, N, kvnn, lamb, kmax=15.0, kmid=3.0, ntot=120,
        generator='Wegner'
    ):
        
        # Instance attributes
        self.nucleus_name = nucleus_name
        self.kvnn = kvnn
        self.lamb = lamb
        
        # Set table of Clebsch-Gordan coefficients
        self.clebsch_gordan = ClebschGordan()
        
        # Set single-particle basis
        self.woods_saxon = WoodsSaxon(nucleus_name, Z, N, self.clebsch_gordan)
        
        # Set \delta U matrix element function
        self.delta_U_matrix_element = DeltaUMatrixElement()

        
    def ipm_term(self, tau, q_array):
        """Compute the I * n(q) * I term."""
        
        I_array = np.zeros_like(q_array)
        theta_array = np.zeros_like(q_array)
        phi_array = np.zeros_like(q_array)
        
        # Loop over spin projections
        for sigma in np.array([1/2, -1/2]):
            
            # Loop over occupied s.p. states
            for alpha in self.woods_saxon.occupied_states_array:
                
                # Single-particle quantum numbers
                n, l, j, m_j, m_t = alpha

                # Single-particle wave function with z-axis along q_vector
                psi_alpha_array = self.woods_saxon.psi(
                    q_array, theta_array, phi_array, sigma, tau, n, l, j, m_j,
                    m_t
                )

                I_array += np.abs(psi_alpha_array) ** 2

        print("Done with I term.")
                    
        return I_array
    
    def delU_term(self, tau, q_array, neval):
        """Compute \delta U * n(q) * I + I * n(q) * \delta U^\dagger terms."""

        # Relative momenta from 0 to 10 fm^-1
        k_limits = [0, 10]
        # C.o.M. momenta up to 3 fm^-1
        K_limits = [0, 3]
        # Polar angle from 0 to \pi
        theta_limits = [0, np.pi]
        # Azimuthal angle from 0 to 2\pi
        phi_limits = [0, 2*np.pi]

        # Set-up integrator with multiple processors
        integ = vegas.Integrator([
            k_limits, theta_limits, phi_limits,
            K_limits, theta_limits, phi_limits
        ], nproc=8)
    
        print("Starting \delta U + \delta U^\dagger term...\n")

        # Evaluate the \delta U + \delta U^\dagger term for each q
        delta_U_array = np.zeros_like(q_array)
        delta_U_errors = np.zeros_like(q_array)
        for i, q in enumerate(q_array):

            # TESTING
            t0 = time.time()
        
            integrand = DeltaUIntegrand(q, tau, self.woods_saxon,
                                        self.delta_U_matrix_element)
        
            # Train the integrator
            integ(integrand, nitn=5, neval=neval)
    
            # Final result
            result = integ(integrand, nitn=10, neval=neval)
        
            delta_U_array[i] = result.mean
            delta_U_errors[i] = result.sdev
        
            # TESTING
            t1 = time.time()
            mins = (t1-t0)/60
            percent = (i+1) / q_array.size * 100
            print(f"{percent:.1f} percent done after {mins:.3f} minutes.")
        
        print("\nDone with \delta U + \delta U^\dagger term.")

        return delta_U_array, delta_U_errors
    
    def delU2_term(self, tau, q_array, neval):
        """Compute the \delta U * n(q) * \delta U^\dagger term."""

        # Relative momenta from 0 to 10 fm^-1
        k_limits = [0, 10]
        # C.o.M. momenta up to 3 fm^-1
        K_limits = [0, 3]
        # Polar angle from 0 to \pi
        theta_limits = [0, np.pi]
        # Azimuthal angle from 0 to 2\pi
        phi_limits = [0, 2*np.pi]

        # Set-up integrator with multiple processors
        integ = vegas.Integrator([
            k_limits, theta_limits, phi_limits,
            k_limits, theta_limits, phi_limits,
            K_limits, theta_limits, phi_limits
        ], nproc=8)
    
        print("Starting \delta U \delta U^\dagger term...\n")

        # Evaluate the \delta U \delta U^\dagger term for each q
        delta_U2_array = np.zeros_like(q_array)
        delta_U2_errors = np.zeros_like(q_array)
        for i, q in enumerate(q_array):

            # TESTING
            t0 = time.time()
        
            integrand = DeltaU2Integrand(q, tau, self.woods_saxon,
                                         self.delta_U_matrix_element)
        
            # Train the integrator
            integ(integrand, nitn=5, neval=neval)
    
            # Final result
            result = integ(integrand, nitn=10, neval=neval)
        
            delta_U2_array[i] = result.mean
            delta_U2_errors[i] = result.sdev
        
            # TESTING
            t1 = time.time()
            mins = (t1-t0)/60
            percent = (i+1) / q_array.size * 100
            print(f"{percent:.1f} percent done after {mins:.3f} minutes.")
        
        print("\nDone with \delta U \delta U^\dagger term.")

        return delta_U2_array, delta_U2_errors
        
    def compute_normalization(self, q_array, q_weights, n_array):
        """Normalization of single-nucleon momentum distribution."""
        
        return 4 * np.pi * np.sum(q_weights * q_array ** 2 * n_array)
    
    def evaluate(
            self, tau, delU_neval, delU2_neval, qmax=10.0, qmid=3.0, ntotq=100,
            nmodq=60, print_normalization=False, save=False
    ):
        """Compute the single-nucleon momentum distribution."""
    
        # Set points in q
        q_array, q_weights = momentum_mesh(qmax, qmid, ntotq, nmod=nmodq)
    
        # Compute the I term
        I_array = self.ipm_term(tau, q_array)
        
        # Compute the \delta U + \delta U^\dagger terms
        delU_array, delU_errors = self.delU_term(tau, q_array, delU_neval)
        
        # Compute the \delta U \delta U^\dagger term
        delU2_array, delU2_errors = self.delU2_term(tau, q_array, delU2_neval)
        
        n_array = I_array + delU_array + delU2_array
        n_errors = np.sqrt(delU_errors ** 2 + delU2_array ** 2)
        
        # Option to print normalization of the total momentum distribution
        if print_normalization:
            normalization = self.compute_normalization(q_array, q_weights,
                                                       n_array)
            if tau == 1/2:
                print(f"Z = {normalization:.5f}.")
            elif tau == -1/2:
                print(f"N = {normalization:.5f}.")
    
        # Option to save the momentum distribution as a .txt file
        if save:
            self.save_momentum_distribution(
                tau, q_array, q_weights, n_array, n_errors, I_array,
                delta_U_array, delta_U_errors, delta_U2_array, delta_U2_errors
            )
    
        return q_array, q_weights, n_array, n_errors
    
    def save_momentum_distribution(
            self, tau, q_array, q_weights, n_array, n_errors, I_array,
            delta_U_array, delta_U_errors, delta_U2_array, delta_U2_errors
    ):
        """Save the momentum distribution along with the isolated contributions."""
    
        data = np.vstack(
            (q_array, q_weights, n_array, n_errors, I_array, delta_U_array,
             delta_U_errors, delta_U2_array, delta_U2_errors)
        ).T
            
        if tau == 1/2:
            nucleon = 'proton'
        elif tau == -1/2:
            nucleon = 'neutron'
                
        hdr = (
            "q, q weight, n(q), n(q) error, I, \delta U + \delta U^\dagger,"
            " \delta U + \delta U^\dagger error, \delta U^2, \delta U^2 error\n"
        )
    
        directory = 'momentum_distributions/'

        file_name = replace_periods(
            f"{self.nucleus_name}_{nucleon}_momentum_distribution"
            f"_kvnn_{self.kvnn}_lamb_{self.lamb}"
        )
    
        np.savetxt(directory + file_name + '.txt', data, header=hdr)
        
    def load_momentum_distribution(self, nucleon):
        """Load and return the momentum distribution along with the isolated
        contributions.
        """
    
        directory = 'momentum_distributions/'

        file_name = replace_periods(
            f"{self.nucleus_name}_{nucleon}_momentum_distribution"
            f"_kvnn_{self.kvnn}_lamb_{self.lamb}"
        )
    
        data = np.loadtxt(directory + file_name + '.txt')

        return data

In [ ]:
# Compare speed of IPM n(q)
md = MomentumDistribution('He4', 2, 2, 6, 1.5)
q_array, q_weights, n_array, n_errors = md.evaluate(1/2,
                                                    print_normalization=True)

In [ ]:
def set_occupied_state_pairs(woods_saxon):
    """List of occupied s.p. state pairs."""
    
    occupied_state_pairs = []
    for alpha in woods_saxon.occupied_states:
        for beta in woods_saxon.occupied_states:
            occupied_state_pairs.append((alpha, beta))
    return occupied_state_pairs

In [ ]:
def set_spin_configurations():
    """List of spin projection configurations."""
    
    spin_projections = [1/2, -1/2]
    spin_configurations = []
    for spin_1 in spin_projections:
        for spin_2 in spin_projections:
            for spin_3 in spin_projections:
                for spin_4 in spin_projections:
                    spin_configurations.append((spin_1, spin_2, spin_3, spin_4))
                        
    return spin_configurations

In [ ]:
class SingleParticleState:
    """
    Single-particle state class. Packs together the following single-particle
    quantum numbers into one object.
    
    Parameters
    ----------
    n : int
        Principal quantum number n = 1, 2, ...
    l : int
        Orbital angular momentum l = 0, 1, ...
    j : float
        Total angular momentum j = 1/2, 3/2, ...
    m_j : float
        Total angular momentum projection m_j = -j, -j+1, ..., j.
    m_t : float
        Isospin projection m_t = 1/2 or -1/2.
    
    """
    
    
    def __init__(self, n, l, j, m_j, m_t):
        
        # Check if m_j is valid
        if abs(m_j) > j:
            raise RuntimeError("m_j is not valid.")
            
        # Check that |m_t| = 1/2
        if abs(m_t) != 1/2:
            raise RuntimeError("m_t is not valid.")
            
        self.n = n
        self.l = l
        self.j = j
        self.m_j = m_j
        self.m_t = m_t
        
        if m_t == 1/2:
            self.nucleon = 'proton'
        elif m_t == -1/2:
            self.nucleon = 'neutron'
        
        
    def __eq__(self, sp_state):

        if (
            self.n == sp_state.n and self.l == sp_state.l
            and self.j == sp_state.j and self.m_j == sp_state.m_j
            and self.m_t == sp_state.m_t
        ):
            
            return True
        
        else:
            
            return False
        
        
    def __str__(self):
        
        # Spectroscopic notation of orbital angular momentum
        l_str = convert_l_to_string(self.l)  # E.g., 's', 'p', 'd', ...
        
        # Display j subscript as a fraction
        numerator = 2*int(self.j) + 1
        denominator = 2

        return fr"${self.n}{l_str}_{{{numerator}/{denominator}}}$"

In [ ]:
class WoodsSaxon_OLD:
    """
    Woods-Saxon orbitals class. Handles the radial wave functions associated
    with the Woods-Saxon potential from the subroutine in woodsaxon.f90. Outputs
    radial functions in coordinate and momentum space.
    
    Parameters
    ----------
    nucleus_name : str
        Name of the nucleus (e.g., 'O16', 'Ca40', etc.)
    Z : int
        Proton number of the nucleus.
    N : int
        Neutron number of the nucleus.
    run_woods_saxon : bool, optional
        Option to run the Woods-Saxon subroutine to generate orbital files.
    n_max : int, optional
        Maximum principal quantum number where n = 1, 2, ..., n_max.
    l_max : int, optional
        Maximum orbital angular momentum where l = 0, 1, ..., l_max.
    rmax : float, optional
        Maximum r for orbital tables.
    ntab : int, optional
        Number of points for orbital tables.
        
    """
    
    
    def __init__(self, nucleus_name, Z, N, cg_table, rmax=40, ntab=2000,
                 kmax=10.0, kmid=2.0, ntot=120):
        
        # Set instance attributes
        self.woods_saxon_directory = f"../data/woods_saxon/{nucleus_name}/"
        self.cg_table = cg_table

        # Order single-particle states with lowest energy first
        self.order_sp_states(Z, N)
        
        # Organize wave functions in dictionary
        self.sp_wfs = {}
        for sp_state in self.sp_states:
            
            # Wave functions are independent of m_j, so fix m_j=j
            if sp_state.m_j == sp_state.j:
                
                file_name = self.get_orbital_file_name(sp_state)
 
                data = np.loadtxt(self.woods_saxon_directory + file_name)
                
                # Use n, l, j, m_t as the key
                key = (sp_state.n, sp_state.l, sp_state.j, sp_state.m_t)
                self.sp_wfs[key] = data[:, 1]
                
        # r_array and dr are the same for every s.p. state
        self.r_array = data[:, 0]
        self.dr = max(self.r_array) / len(self.r_array)
        
        # Interpolate occupied s.p. wave functions w.r.t. momentum k
        self.interpolate_wavefunctions(kmax, kmid, ntot)
        
    def get_orbital_file_name(self, sp_state):
        """Returns the file name of the orbital."""
            
        # Proton
        if sp_state.m_t == 1/2:
            file_name = (f"p.n{int(sp_state.n-1)}.l{int(sp_state.l)}"
                         f".j{int(2*sp_state.j)}.orb")
        # Neutron
        elif sp_state.m_t == -1/2:
            file_name = (f"n.n{int(sp_state.n-1)}.l{int(sp_state.l)}"
                         f".j{int(2*sp_state.j)}.orb")
            
        return file_name
        
    def order_sp_states(self, Z, N):
        """Keep track of all s.p. states and occupied s.p. states"""

        self.sp_states = []  # All single-particle states
        self.occupied_states = []  # Occupied single-particle states < E_F
        proton_count = 0
        neutron_count = 0
        
        # File with details of the orbitals
        ws_file = self.woods_saxon_directory + "ws_log"
    
        # Order single-particle states using the ws_log file
        with open(ws_file, 'r') as f:
            for line in f:
                unit = line.strip().split()
                
                # Protons
                if len(unit) > 0 and unit[0] == '1':

                    j = int(unit[3])/2
                    for m_j in np.arange(-j, j+1, 1):
                        sp_state = SingleParticleState(
                            int(unit[1])+1, int(unit[2]), j, m_j, 1/2
                        )  # n, l, j, m_j, m_t
                    
                        self.sp_states.append(sp_state)
                    
                        if proton_count < Z:
                            self.occupied_states.append(sp_state)
                            # Add up filled proton states
                            proton_count += 1
                    
                
                # Neutrons
                elif len(unit) > 0 and unit[0] == '2':

                    j = int(unit[3])/2
                    for m_j in np.arange(-j, j+1, 1):
                        sp_state = SingleParticleState(
                            int(unit[1])+1, int(unit[2]), j, m_j, -1/2
                        )  # n, l, j, m_j, m_t
                    
                        self.sp_states.append(sp_state)
                    
                        if neutron_count < N:
                            self.occupied_states.append(sp_state)
                            # Add up filled neutron states
                            neutron_count += 1

    def get_wf_rspace(self, sp_state, print_normalization=False):
        """Single-particle wave function in coordinate space."""
        
        # n, l, j, m_t
        key = (sp_state.n, sp_state.l, sp_state.j, sp_state.m_t) 
        u_array = self.sp_wfs[key]

        # Normalization: \int dr |u(r)|^2 = 1
        if print_normalization:
            normalization = np.sum(self.dr * u_array ** 2)
            print(f"Coordinate space normalization = {normalization}.")

        return self.r_array, u_array
    
    def fourier_transformation(self, l, r_array, k_array):
        """Fourier transformation matrix for given orbital angular momentum."""
        
        # r_array column vectors and k_array row vectors where both grids are
        # n x m matrices
        r_cols, k_rows = np.meshgrid(r_array, k_array)
        
        # Transformation matrix with shape n x m, where m is the length of
        # r_array and n is the length of the k_array
        M = 1j ** (-l) * np.sqrt(2/np.pi) * self.dr * r_cols * spherical_jn(
            l, k_rows*r_cols
        )
        
        return M
    
    
    def get_wf_kspace(
            self, sp_state, kmax, kmid, ntot, print_normalization=False,
            interpolate=False
    ):
        """Single-particle wave function in momentum space."""
    
        # Set momentum mesh with more points at low momentum
        k_array, k_weights = momentum_mesh(kmax, kmid, ntot)
    
        # Get coordinate-space s.p. wave function
        r_array, u_array = self.get_wf_rspace(sp_state)

        # Fourier-transform the wave function to momentum space
        phi_array = (self.fourier_transformation(sp_state.l, r_array, k_array)
                     @ u_array)
    
        # Normalization: \int dk k^2 |\phi(k)|^2 = 1
        if print_normalization:
            normalization = np.sum(k_weights * k_array ** 2
                                   * abs(phi_array) ** 2)
            print(f"Momentum space normalization = {normalization}.")
            
        # Interpolate and return function?
        if interpolate:
            
            if sp_state.l % 2 == 0:  # Even l
                
                phi_func = InterpolatedUnivariateSpline(k_array, phi_array.real)
                
            else:  # Odd l
            
                phi_func = InterpolatedUnivariateSpline(k_array, phi_array.imag)
                
            return phi_func
        
        # Otherwise return momentum, weights, and \phi(k)
        else:
            return k_array, k_weights, phi_array
        
        
    def interpolate_wavefunctions(self, kmax, kmid, ntot):
        """Create dictionary of \phi(k) interpolated functions where the key
        is the single-particle state.
        """
        
        # Organize interpolated wave functions in dictionary with s.p. quantum
        # numbers as the key
        self.phi_functions = {}
        
        for sp_state in self.occupied_states:
            
            key = (sp_state.n, sp_state.l, sp_state.j, sp_state.m_t)
            self.phi_functions[key] = self.get_wf_kspace(sp_state, kmax, kmid,
                                                         ntot, interpolate=True)
            
    
    def psi(self, sp_state, k, theta, phi, sigma, tau):
        """Single-particle wave function \psi_\alpha(k_vector; \sigma, \tau)."""

        # Calculate \phi_\alpha(k)
        key = (sp_state.n, sp_state.l, sp_state.j, sp_state.m_t)
        if sp_state.l % 2 == 0:  # Even l
            phi_sp_wf = self.phi_functions[key](k)
        else:  # Odd l needs factor of i^-l
            phi_sp_wf = 1j ** (-sp_state.l) * self.phi_functions[key](k)
    
        # Calculate spinor spherical harmonic
        Y_jml = self.spinor_spherical_harmonic(sp_state.l, sp_state.j,
                                               sp_state.m_j, theta, phi, sigma)
    
        # Isospinor indexed by \tau \chi_{m_t}(\tau)
        chi_tau = kronecker_delta(tau, sp_state.m_t)

        return phi_sp_wf * Y_jml * chi_tau
    

    def spinor_spherical_harmonic(self, l, j, m_j, theta, phi, sigma):
        """Spinor spherical harmonic for a s.p. state described by the quantum
        numbers j, m_j, l, and s=1/2.
        """
        
        # Spinor indexed by \sigma \eta_{m_s}^(\sigma) = \delta_{m_s, \sigma}
        m_s = sigma
    
        # m_l must be fixed since m_j and m_s are determined
        m_l = m_j - m_s
        
        # Check that |m_l| <= l
        if np.abs(m_l) <= l:
        
            # Clebsch-Gordan coefficient for l-s coupling
            cg = self.cg_table[(l, m_l, 1/2, m_s, j, m_j)]
        
            # Spherical harmonic
            Y_lm = sph_harm(m_l, l, phi, theta)
            
            return cg * Y_lm
        
        else:
            
            return 0+0j

# Partial wave channel class

In [ ]:
# TODO: Probably don't need this, move down
class PartialWaveChannel:
    """
    Partial wave channel class. Packs together the quantum numbers of a partial
    wave channel into one object.
    
    Parameters
    ----------
    channel : str
        Name of the channel (e.g., '1S0').
    
    """

    
    def __init__(self, channel):
    
        # Set instance attributes
        self.channel = channel
        self.L, self.Lp = self.get_orbital_angular_momentum()
        self.J = self.get_angular_momentum()
        self.S = self.get_spin()
        self.T = self.get_isospin(self.L, self.S)

        
    def __eq__(self, channel):

        if (
            self.L == channel.L and self.Lp == channel.Lp
            and self.J == channel.J and self.S == channel.S
            and self.T == channel.T
        ):
            
            return True
        
        else:
            
            return False
    
    
    def get_orbital_angular_momentum(self):
        """Gets the total orbital angular momentum L and L'."""
        
        if self.channel[1] == 'S':
            L = 0
        elif self.channel[1] == 'P':
            L = 1
        elif self.channel[1] == 'D':
            L = 2
        elif self.channel[1] == 'F':
            L = 3
        elif self.channel[1] == 'G':
            L = 4
        elif self.channel[1] == 'H':
            L = 5
        else:
            raise RuntimeError("Channel L exceeds the range of the function.")
            
        # Coupled-channel
        if coupled_channel(self.channel[:3]):
            
            if self.channel[5] == 'S':
                Lp = 0
            elif self.channel[5] == 'P':
                Lp = 1
            elif self.channel[5] == 'D':
                Lp = 2
            elif self.channel[5] == 'F':
                Lp = 3
            elif self.channel[5] == 'G':
                Lp = 4
            elif self.channel[5] == 'H':
                Lp = 5
            else:
                raise RuntimeError(
                    "Channel L' exceeds the range of the function.")
        
        # L = L' if the channel is not coupled
        else:
            
            Lp = L
            
        return L, Lp
    
    
    def get_angular_momentum(self):
        """Total angular momentum J = 0, 1, 2, ..."""
        
        J = int(self.channel[2])
        
        return J
    
    
    def get_spin(self):
        """Total spin S = 0 or 1."""
        
        S = int((int(self.channel[0])-1)/2)
        
        return S
    
    
    def get_isospin(self, L, S):
        """Total isospin according to antisymmetry."""
    
        # Make sure [1-(-1)^(L+S+T)] is not zero
        if (1 - (-1) ** (L+S)) == 0:
            T = 1
        else:
            T = 0
        
        return T

# $\delta U$ plane-wave matrix element class

In [ ]:
# Try interpolating using your own Interpolate2D and then setting
# - self.delta_U_1S0 (a function that works in JAX)
# - self.delta_U_3S1_3S1
# - ...

# How to loop over partial wave channels?

In [ ]:
class DeltaUMatrixElement:
    """DOCSTRING"""
    
    def __init__(self, cg_table, kvnn, kmax, kmid, ntot, generator, lamb,
                 channels):
        
        # Set instance attributes
        self.cg_table = cg_table
        self.channels = channels
        
        # Set spin singlet and triplet channels
        self.spin_singlet_channels = self.get_singlet_channels()
        self.spin_triplet_channels = self.get_triplet_channels()
        
        # Set \delta U and \delta U^\dagger functions
        self.get_delta_U_functions(kvnn, kmax, kmid, ntot, generator, lamb)
    
    
    def __call__(
            self, k, theta_k, phi_k, kp, theta_kp, phi_kp, sigma_1, sigma_2,
            sigma_3, sigma_4, tau_1, tau_2, tau_3, tau_4, hc=False
    ):
        """Sum over partial waves to evaluate \delta U matrix element in
        plane-wave basis.
        """
        
        matrix_element = 0+0j
        
        # Check that M_T is conserved
        if tau_1 + tau_2 != tau_3 + tau_4:
            return matrix_element
        else:
            M_T = tau_1 + tau_2
            
        # Set total spin projections
        M_S = sigma_1 + sigma_2
        M_Sp = sigma_3 + sigma_4

        # Loop over only spin triplet channels
        if np.abs(M_S) == 1 or np.abs(M_Sp) == 1:
            channels = self.spin_triplet_channels
        # Loop over only spin singlet channels
        elif np.abs(M_T) == 1:
            channels = self.spin_singlet_channels
        # Loop over all channels
        else:
            channels = self.channels
            
        for channel in channels:

            # Get quantum numbers of channel
            pwc = PartialWaveChannel(channel)
            L, Lp, S, J, T = pwc.L, pwc.Lp, pwc.S, pwc.J, pwc.T
            
            # Check that |M_T| <= T and |M_S|, |M_S'| <= S
            if np.abs(M_T) <= T and np.abs(M_S) <= S and np.abs(M_Sp) <= S:
                
                # Get spin Clebsch-Gordan coefficients
                spin_12_cg = self.cg_table[(1/2, sigma_1, 1/2, sigma_2, S, M_S)]
                spin_34_cg = self.cg_table[(1/2, sigma_3, 1/2, sigma_4, S,
                                            M_Sp)]
    
                # Get isospin Clebsch-Gordan coefficients
                isospin_12_cg = self.cg_table[(1/2, tau_1, 1/2, tau_2, T, M_T)]
                isospin_34_cg = self.cg_table[(1/2, tau_3, 1/2, tau_4, T, M_T)]
                
                # \delta U^\dagger in partial-wave basis
                key = (L, Lp, J, S, T)
                if hc:
                    delta_U_partial_wave = self.delUdag_functions[key].ev(k, kp)
                # \delta U in partial-wave basis
                else:
                    delta_U_partial_wave = self.delU_functions[key].ev(k, kp)
            
                # Sum over M_J
                for M_J in np.arange(-J, J+1):
                    
                    # Set total orbital angular momentum projection
                    M_L = M_J - M_S
                    M_Lp = M_J - M_Sp
                    
                    # Check |M_L| <= L conditions
                    if np.abs(M_L) <= L and np.abs(M_Lp) <= Lp:
                        
                        # Get L-S coupling Clebsch-Gordan coefficients
                        lsj_cg = self.cg_table[(L, M_L, S, M_S, J, M_J)]
                        lpsj_cg = self.cg_table[(Lp, M_Lp, S, M_Sp, J, M_J)]
                        
                        # Calculate spherical harmonics
                        Y_k = sph_harm(M_L, L, phi_k, theta_k)
                        Y_kp = sph_harm(M_Lp, Lp, phi_kp, theta_kp)

                        # Factors of [1-(-1)^(L+S+T)] [1-(-1)^(L'+S+T)] included
                        matrix_element += (
                            1/2 * 2/np.pi * 4 * spin_12_cg * spin_34_cg
                            * isospin_12_cg * isospin_34_cg
                            * delta_U_partial_wave * lsj_cg * lpsj_cg
                            * Y_k * np.conj(Y_kp)
                        )
    
        return matrix_element
    
    
    def interpolate_delta_U(self, channel, potential, generator, lamb,
                            hc=False):
        """Interpolate \delta U(k, k') for the given channel."""

        # Get momentum mesh
        kmax, kmid, ntot = potential.kmax, potential.kmid, potential.ntot
        k_array, k_weights = momentum_mesh(kmax, kmid, ntot)
    
        # Get SRG transformation with integration weights [unitless]
        U_matrix_weights = load_srg_transformation(potential, generator, lamb)

        # Calculate \delta U = U - I
        I_matrix_weights = np.eye(len(U_matrix_weights))
        if hc:  # Hermitian conjugate
            delU_matrix_weights = (U_matrix_weights - I_matrix_weights).T
        else:
            delU_matrix_weights = U_matrix_weights - I_matrix_weights

        # Get specific sub-block if coupled-channel and unattach weights
        if channel in ['3S1-3D1', '3P2-3F2', '3D3-3G3']:
            delU_matrix = unattach_weights_from_matrix(
                k_array, k_weights, delU_matrix_weights[:ntot,ntot:]
            )
        elif channel in ['3D1-3S1', '3F2-3P2', '3G3-3D3']:
            delU_matrix = unattach_weights_from_matrix(
                k_array, k_weights, delU_matrix_weights[ntot:,:ntot]
            )
        elif channel in ['3D1-3D1', '3F2-3F2', '3G3-3G3']:
            delU_matrix = unattach_weights_from_matrix(
                k_array, k_weights, delU_matrix_weights[ntot:,ntot:]
            )
        else:
            delU_matrix = unattach_weights_from_matrix(
                k_array, k_weights, delU_matrix_weights[:ntot,:ntot]
            )
        
        # Interpolate \delta U(k, k')
        delU_func = RectBivariateSpline(k_array, k_array, delU_matrix)

        return delU_func
        
        
    def get_delta_U_functions(self, kvnn, kmax, kmid, ntot, generator, lamb):
        """Get \delta U and \delta U^\dagger functions."""
        
        self.delU_functions = {}
        self.delUdag_functions = {}
        for channel in self.channels:
        
            # Set channel argument to be compatible with potential functions
            if channel[:3] == '3D1':
                channel_arg = '3S1'
            elif channel[:3] == '3F2':
                channel_arg = '3P2'
            elif channel[:3] == '3G3':
                channel_arg == '3D3'
            else:
                channel_arg = channel[:3]
            
            # Potential in specified partial wave channel
            potential = Potential(kvnn, channel_arg, kmax, kmid, ntot)
            
            # Use partial wave channel as key
            pwc = PartialWaveChannel(channel)
            key = (pwc.L, pwc.Lp, pwc.J, pwc.S, pwc.T)
        
            # Set \delta U and \delta U^\dagger functions
            self.delU_functions[key] = self.interpolate_delta_U(
                channel, potential, generator, lamb)
            self.delUdag_functions[key] = self.interpolate_delta_U(
                channel, potential, generator, lamb, hc=True)
            
            
    def get_singlet_channels(self):
        """Set all S=0 partial wave channels."""
        
        spin_singlet_channels = []
        for channel in self.channels:
            pwc = PartialWaveChannel(channel)
            if pwc.S == 0:
                spin_singlet_channels.append(channel)
        
        return spin_singlet_channels
                
        
    def get_triplet_channels(self):
        """Set all S=1 partial wave channels."""
        
        spin_triplet_channels = []
        for channel in self.channels:
            pwc = PartialWaveChannel(channel)
            if pwc.S == 1:
                spin_triplet_channels.append(channel)         
        
        return spin_triplet_channels

In [ ]:
# Don't worry about \delta U \delta U^\dagger yet

# $\delta U$ term integrand class

In [ ]:
class DeltaUIntegrand:
    """Evaluates the integrand of the \delta U + \delta U^\dagger term."""
    
    
    def __init__(self, q, tau, woods_saxon, delta_U_matrix_element,
                 spin_configurations, occupied_state_pairs):
        
        # Set momenta and isospin as instance attributes
        self.q = q
        self.tau = tau
        self.isospin_configurations = []
        for taup in [1/2, -1/2]:
            for tau_1 in [1/2, -1/2]:
                for tau_2 in [1/2, -1/2]:
                    if tau + taup == tau_1 + tau_2:
                        self.isospin_configurations.append(
                            (taup, tau_1, tau_2)
                        )
        
        # Set Woods-Saxon and \delta U matrix element classes as instance attributes
        self.woods_saxon = woods_saxon
        self.delta_U_matrix_element = delta_U_matrix_element
        
        # Possible pairs of spin projections
        self.spin_configurations = spin_configurations
        self.N_spin = len(spin_configurations)
    
        # Possible pairs of occupied s.p. states
        self.occupied_state_pairs = occupied_state_pairs
        self.N_occupied_states = len(occupied_state_pairs)
    
    
    def __call__(self, x_array):
        """This method assumes sampling over the following variables:
        
            k, theta_k, phi_k -> vector momenta k,
            K, theta_K, phi_K -> vector momenta K,
            spin projections -> sigma_1, sigma_2, sigma, sigma',
            s.p. state pairs -> \alpha, \beta,
            
        where x_array contains each variable meaning x_array.shape = (8,).
        """
        
        # Choose z-axis to be along q_vector
        q_vector = np.array([0, 0, self.q])
        
        # Relative momenta k
        k, theta_k, phi_k = x_array[0:3]
        k_vector = build_vector(k, theta_k, phi_k)
        
        # C.o.M. momenta K
        K, theta_K, phi_K = x_array[3:6]
        K_vector = build_vector(K, theta_K, phi_K)
        
        # Spin projections \sigma_1, \sigma_2, \sigma, \sigma'
        sigma_1, sigma_2, sigma, sigmap = self.get_spin_configuration(
            x_array[6])
        
        # S.p. state pairs \alpha, \beta < F
        alpha, beta = self.get_occupied_state_pair(x_array[7])
        
        # Calculate vector q - K/2
        qK_vector = q_vector - K_vector/2
        qK, theta_qK, phi_qK = get_vector_components(qK_vector)
        
        # Calculate vector k_1 = K/2 + k
        k1_vector = K_vector/2 + k_vector
        k1, theta_k1, phi_k1 = get_vector_components(k1_vector)
        
        # Calculate vector k_2 = K/2 - k
        k2_vector = K_vector/2 - k_vector
        k2, theta_k2, phi_k2 = get_vector_components(k2_vector)
        
        # Calculate vector K - q
        Kq_vector = K_vector - q_vector
        Kq, theta_Kq, phi_Kq = get_vector_components(Kq_vector)
        
        # Calculate the Jacobian determinant
        jacobian = k ** 2 * np.sin(theta_k) * K ** 2 * np.sin(theta_K)
        
        # \psi_\alpha(q; \sigma, \tau)
        psi_alpha_q = self.woods_saxon.psi(alpha, self.q, 0, 0, sigma, self.tau)
        
        # \psi_\beta(q; \sigma, \tau)
        psi_beta_q = self.woods_saxon.psi(beta, self.q, 0, 0, sigma, self.tau)
        
        # Sum over \tau'
        integrand = 0+0j
        for isospin_projections in self.isospin_configurations:
            
            taup, tau_1, tau_2 = isospin_projections
                
            # Plane-wave matrix elements of \delta U and \delta U^\dagger
            delta_U_plane_wave = self.delta_U_matrix_element(
                k, theta_k, phi_k, qK, theta_qK, phi_qK, sigma_1, sigma_2,
                sigma, sigmap, tau_1, tau_2, self.tau, taup, hc=False
            )
            delta_U_dag_plane_wave = self.delta_U_matrix_element(
                qK, theta_qK, phi_qK, k, theta_k, phi_k, sigma, sigmap, sigma_1,
                sigma_2, self.tau, taup, tau_1, tau_2, hc=True
            )
        
            # \psi_\alpha(K/2+k; \sigma_1, \tau_1)
            psi_alpha_1 = self.woods_saxon.psi(alpha, k1, theta_k1, phi_k1,
                                               sigma_1, tau_1)
        
            # \psi_\beta(K/2-k; \sigma_2, \tau_2)
            psi_beta_2 = self.woods_saxon.psi(beta, k2, theta_k2, phi_k2,
                                              sigma_2, tau_2)

            # \psi_\beta(K-q; \sigma', \tau')
            psi_beta_Kq = self.woods_saxon.psi(beta, Kq, theta_Kq, phi_Kq,
                                               sigmap, taup)
        
            # \psi_\alpha(K-q; \sigma', \tau')
            psi_alpha_Kq = self.woods_saxon.psi(alpha, Kq, theta_Kq, phi_Kq,
                                                sigmap, taup)
        
            # \delta U term
            delta_U_term = (
                delta_U_plane_wave * np.conj(psi_alpha_1) * np.conj(psi_beta_2)
                * (psi_beta_Kq * psi_alpha_q - psi_alpha_Kq * psi_beta_q)
            )
        
            # \delta U^\dagger term
            delta_U_dag_term = (
                delta_U_dag_plane_wave * psi_alpha_1 * psi_beta_2
                * (
                    np.conj(psi_beta_Kq) * np.conj(psi_alpha_q)
                    - np.conj(psi_alpha_Kq) * np.conj(psi_beta_q)
                )
            )

            # Add together for full integrand
            integrand += (
                1/2 * jacobian * self.N_spin * self.N_occupied_states * (
                    delta_U_term + delta_U_dag_term
                )
            )
        
        return integrand.real    
    
    
    def get_spin_configuration(self, x):
        """Given a number between 0 and 1, return a set of four spin
        projections.
        """
        
        index = np.floor(x * self.N_spin).astype(int)
        return self.spin_configurations[index]
    
    
    def get_occupied_state_pair(self, x):
        """Given a number between 0 and 1, return a pair of occupied quantum
        states.
        """
        
        index = np.floor(x * self.N_occupied_states).astype(int)
        return self.occupied_state_pairs[index]

# Momentum distribution functions

In [ ]:
def compute_I_term(q_array, tau, woods_saxon):
    """Compute the I * n(q) * I term."""
        
    I_array = np.zeros_like(q_array)
    theta_array = np.zeros_like(q_array)
    phi_array = np.zeros_like(q_array)
        
    # Loop over spin projections
    for sigma in np.array([1/2, -1/2]):
            
        # Loop over occupied s.p. states
        for alpha in woods_saxon.occupied_states:

            # Single-particle wave function with z-axis along q_vector
            psi_alpha_array = woods_saxon.psi(alpha, q_array, theta_array,
                                              phi_array, sigma, tau)

            I_array += np.abs(psi_alpha_array) ** 2
                    
    return I_array

In [ ]:
def compute_delta_U_term(q_array, tau, woods_saxon, delta_U_matrix_element,
                         spin_configurations, occupied_state_pairs, neval):
    """Compute the \delta U * n(q) * I + I * n(q) * \delta U^\dagger terms."""

    # Relative momenta from 0 to 10 fm^-1
    k_limits = [0, 10]
    # C.o.M. momenta up to 3 fm^-1
    K_limits = [0, 3]
    # Polar angle from 0 to \pi
    theta_limits = [0, np.pi]
    # Azimuthal angle from 0 to 2\pi
    phi_limits = [0, 2*np.pi]
    # Discrete variables for \alpha, \beta, and spin projections
    dv_limits = [0, 1]

    # Set-up integrator with multiple processors
    # IN A JUPYTER NOTEBOOK YOU HAVE TO USE nproc=1!
    integ = vegas.Integrator([
        k_limits, theta_limits, phi_limits,
        K_limits, theta_limits, phi_limits,
        dv_limits, dv_limits
    ], nproc=1)

    # Evaluate the \delta U + \delta U^\dagger term for each q
    delta_U_array = np.zeros_like(q_array)
    delta_U_errors = np.zeros_like(q_array)
    for i, q in enumerate(q_array):

        # TESTING
        t0 = time.time()
        
        integrand = DeltaUIntegrand(
            q, tau, woods_saxon, delta_U_matrix_element, spin_configurations,
            occupied_state_pairs
        )
        
        # Train the integrator
        integ(integrand, nitn=5, neval=neval)
    
        # Final result
        result = integ(integrand, nitn=10, neval=neval)
        
        delta_U_array[i] = result.mean
        delta_U_errors[i] = result.sdev
        
        # TESTING
        t1 = time.time()
        mins = (t1-t0)/60
        print(f"Done with q={q:.4f} after {mins:.4f} minutes.")

    return delta_U_array, delta_U_errors

In [ ]:
def compute_delta_U2_term():
    return None

In [ ]:
def compute_normalization(q_array, q_weights, n_array):
    """Compute the normalization of the momentum distribution."""

    return 4*np.pi * np.sum(q_weights * q_array ** 2 * n_array)

In [ ]:
def compute_momentum_distribution(
        nucleus_name, Z, N, tau, kvnn, lamb, channels, kmax=15.0, kmid=3.0,
        ntot=120, generator='Wegner', neval=5e4, print_normalization=False,
        save=False
):
    """Compute the single-nucleon momentum distribution."""
    
    # Set table of Clebsch-Gordan coefficients
    jmax = 5  # This should cover nuclei as heavy as Pb208
    cg_table = compute_clebsch_gordan_table(jmax)
    
    # Set single-particle basis
    woods_saxon = WoodsSaxon(nucleus_name, Z, N, cg_table)
    # Get pairs of occupied states
    occupied_state_pairs = set_occupied_state_pairs(woods_saxon)

    # Set points in q
    qmax, qmid, ntot_q, nmod_q = 10.0, 2.0, 100, 50
    q_array, q_weights = momentum_mesh(qmax, qmid, ntot_q, nmod=nmod_q)
    
    # Compute the I term
    I_array = compute_I_term(q_array, tau, woods_saxon)
    
    # Get sets of four spin projection configurations
    spin_configurations = set_spin_configurations()
    
    # Initialize \delta U matrix element class
    delta_U_matrix_element = DeltaUMatrixElement(
        cg_table, kvnn, kmax, kmid, ntot, generator, lamb, channels)
    
    # Compute the \delta U + \delta U^\dagger term
    delta_U_array, delta_U_errors = compute_delta_U_term(
        q_array, tau, woods_saxon, delta_U_matrix_element, spin_configurations,
        occupied_state_pairs, neval
    )
    
    # Compute the \delta U \delta U^\dagger term
    delta_U2_array = np.zeros_like(I_array)
    delta_U2_errors = np.zeros_like(I_array)
    # Don't worry about \delta U \delta U^\dagger yet
    
    # Combine each term for the total momentum distribution [fm^3]
    n_array = I_array + delta_U_array + delta_U2_array
    n_errors = np.sqrt(delta_U_errors ** 2 + delta_U2_errors ** 2)

    # Option to print normalization of the total momentum distribution
    if print_normalization:
        normalization = compute_normalization(q_array, q_weights, n_array)
        print(f"Normalization = {normalization:.5f}.")
    
    # Option to save the momentum distribution as a .txt file
    if save:
        pass
    
    return q_array, q_weights, n_array, n_errors

In [ ]:
nucleus_name, Z, N = 'He4', 2, 2
tau = 1/2
kvnn = 6
lamb = 1.5
channels = ('1S0', '3S1-3S1', '3S1-3D1', '3D1-3S1', '3D1-3D1')

q_array, q_weights, n_array, n_errors = compute_momentum_distribution(
    nucleus_name, Z, N, tau, kvnn, lamb, channels, print_normalization=True,
    save=False
)

In [ ]:
@jit
def test_func(x, a, b):
    return x * a ** 2 + b

In [ ]:
# Do sum manually
a_array = jnp.arange(0, 5)
b_array = jnp.arange(0, 5)

x = 0.5
value = 0
for a in a_array:
    for b in b_array:
        value += test_func(x, a, b)

print(value)

In [ ]:
# Use vmap
@jit
def vmap_test_func(x, a, b):
    return vmap(test_func, in_axes=(None, 0, 0))(x, a, b)

In [ ]:
a_grid, b_grid = jnp.meshgrid(a_array, b_array, indexing='ij')
value = jnp.sum(vmap_test_func(x, a_grid, b_grid))
print(value)

In [ ]:
@jit
def test_func2(x, gamma):
    a, b = gamma
    return x * a ** 2 + b

In [ ]:
# Do sum manually
a_array = jnp.arange(0, 5)
b_array = jnp.arange(0, 5)
gamma_list = []

for a in a_array:
    for b in b_array:
        gamma_list.append((a, b))
        
gamma_array = jnp.array(gamma_list)

value = 0
for g in gamma_array:
    value += test_func2(x, g)
print(value)

In [ ]:
@jit
def vmap_test_func2(x, gamma):
    return vmap(test_func2, in_axes=(None, 0))(x, gamma)

In [ ]:
value = jnp.sum(vmap_test_func2(x, gamma_array))
print(value)

In [ ]:
a_array = jnp.array([
    [1, 0, 1/2, 1/2, 1/2], [1, 0, 1/2, -1/2, 1/2],
    [1, 0, 1/2, 1/2, -1/2], [1, 0, 1/2, -1/2, -1/2]
])

b_array = jnp.array([
    [1, 0, 1/2, 1/2, 1/2], [1, 0, 1/2, -1/2, 1/2],
    [1, 0, 1/2, 1/2, -1/2], [1, 0, 1/2, -1/2, -1/2]
])
s_array = jnp.array([1/2, -1/2])

In [ ]:
@jit
def test_func3(x, a, b, s1, s2):
    n_a, l_a, j_a, m_j_a, m_t_a = a
    n_b, l_b, j_b, m_j_b, m_t_b = b
    return (
        x * (n_a + n_b) + l_a * j_a + l_b * j_b + m_j_a - m_t_a + m_j_b/2 
        - m_t_b/2 + s1/4 - s2/4
    )

In [ ]:
# Sum over \alpha, \beta, s1, and s2
x = 0.5
value = 0
for a in a_array:
    for b in b_array:
        for s1 in s_array:
            for s2 in s_array:
                value += test_func3(x, a, b, s1, s2)
print(value)

In [ ]:
@jit
def test_func4(x, g, spin_config):
    a, b = g
    n_a, l_a, j_a, m_j_a, m_t_a = a
    n_b, l_b, j_b, m_j_b, m_t_b = b
    s1, s2 = spin_config
    return (
        x * (n_a + n_b) + l_a * j_a + l_b * j_b + m_j_a - m_t_a + m_j_b/2 
        - m_t_b/2 + s1/4 - s2/4
    )

In [ ]:
# Do sum manually
gamma_array = jnp.zeros((2, 5, 4))
spin_list = []

for i, a in enumerate(a_array):
    for j, b in enumerate(b_array):
        gamma_array = gamma_array.at[0, :, i].set(
gamma_array = jnp.array(gamma_list)
for s1 in s_array:
    for s2 in s_array:
        spin_list.append([s1, s2])
spin_config = jnp.array(spin_list)

value = 0
for g in gamma_array:
    value += test_func4(x, gamma_array, spin_config)
print(value)